In [1]:
pip install -U botocore boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json

In [4]:
import json
import numpy as np

def get_embedding(sent, bedrock_runtime, embed_llm_id):
    body = json.dumps({"inputText": sent})
    #claude_prompt = f"\n\nHuman:{sent}\n\nAssistant:"
    #body = json.dumps({ "prompt": claude_prompt, "temperature": 0.5, "top_p": 1, "top_k": 250, "max_tokens_to_sample": 200, "stop_sequences": ["\n\nHuman:"] })
    #print(body)
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=embed_llm_id, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    embedding = response_body.get("embedding")

    return np.squeeze(np.array(embedding).reshape(1, -1)).tolist()


In [6]:
payload = """
99\n2022 ANNUAL REPORT\nShareholders’ Rights\nTo safeguard the Shareholders’ interests and rights, separate resolutions are proposed at the Shareholders’ meetings on \neach substantial issue, including the election of individual directors, for the Shareholders’ consideration and voting. All resolutions put forward at the Shareholders’ meetings will be voted by poll pursuant to the Listing Rules and poll results will be posted on the websites of the Company (www.mi.com) and the Stock Exchange after each Shareholders’ meeting.\nPursuant to the Articles of the Company, extraordinary general meetings shall be convened on the written requisition \nof any one or more Shareholders holding, as of the date of deposit of the requisition, in aggregate shares representing not less than one-tenth of the paid up capital of the Company which carry the right of voting at general meetings of the Company. A written requisition shall be deposited at the principal office of the Company in Hong Kong to the Board or the joint company secretaries for the purpose of requiring an extraordinary general meeting to be called by the Board for the transaction of any business specified in such requisition. Such meeting shall be held within two months after the deposit of such requisition. If within 21 days of such deposit, the Board fails to proceed to convene such meeting, the requisitionist(s) themselves may convene the general meeting in the same manner, and all reasonable expenses \nincurred by the requisitionist(s) as a result of the failure of the Board shall be reimbursed to them by the Company.\nFor the avoidance of doubt, Shareholders must deposit and send the original duly signed written requisition, notice \nor statement (as the case may be) to the Company’s principal place of business in Hong Kong and provide their full name, contact details and identification in order to give effect thereto. Shareholders’ information may be disclosed as required by law.\nThere is no provision allowing the Shareholders to move new resolutions at general meetings under the Cayman \nIslands Companies Law or the Articles. Shareholders who wish to move a resolution may request the Company to \nconvene a general meeting following the procedures set out in the preceding paragraph.\nWith respect to the Shareholders’ right in proposing persons for election as Directors, please refer to the procedures \navailable on the website of the Company.\nDuring the Reporting Period, the Company has amended its Memorandum and Articles of Association by way of a \nspecial resolution passed on June 2, 2022. Details of the amendments are set out in the circular dated April 27, 2022 to the Shareholders.\nThe up-to-date version of the Memorandum and Articles of Association is available on the websites of the Company \nand the Stock Exchange.\nEvents after the Reporting Period\nSave as disclosed in this Corporate Governance Report, there has been no other significant event subsequent to the Reporting Period and up to the Latest Practicable Date that might affect the Group. 
"""

#### Maker sure the name of embedding model is correct

In [7]:
import boto3

runtime_client = boto3.client('bedrock-runtime')
embed_llm_id = "amazon.titan-embed-text-v1" 
#embed_llm_id = "amazon.titan-embed-g1-text-02" 
embedding = get_embedding(payload, runtime_client,embed_llm_id)
print(embedding)

[0.6256718, -0.35734725, -0.40168443, -0.20735937, 0.15836485, 0.017136496, 0.2070499, -3.0146282e-05, 0.52878684, 0.021742512, -0.5155504, -0.21450788, 0.01923035, -0.046604425, 0.57468605, -0.08841933, 0.25399333, 0.2644805, 0.11539299, 0.5880897, -0.43819666, -0.48626593, 0.13130723, 0.15841928, 0.045008957, 0.46990693, 0.07960539, -0.31604704, -0.099142864, 0.15486836, -0.26730445, -0.4499403, -0.2720193, -0.10628982, 0.27215615, -0.2568546, -0.25047895, -0.21775478, 0.09435336, 0.08260973, 0.12849882, 0.129323, 1.1699343, -0.51922023, 0.13417394, -0.7264008, 0.46359974, -0.49007267, 0.14118151, -0.21333848, -0.24426502, 0.15219447, 0.57237756, -0.22197825, -0.2374291, 0.0017012078, -0.4369775, -0.12577751, 0.27849448, -0.08443844, 0.1128023, -0.19750589, -0.24649185, 0.04030031, -0.17576882, 0.06976513, 0.27214372, -0.23255873, 0.276983, -0.15666053, 0.1984599, -0.20750397, 0.38199145, -0.26036274, 0.3652966, -0.0009299114, 0.005312003, 0.16163665, 0.3942949, 0.24980098, -0.400751

#### Please note that certain embedding models may have a shorter input token size, please make sure that the chunk size is relevant to your embedding model

In [18]:
def chunk_words(sequence, chunk_size):
    sequence = sequence.split()
    return [' '.join(sequence[i:i+chunk_size]) for i in range(0, len(sequence), chunk_size)]

def query_endpoint(payload, runtime_client,embed_llm_id):
    embeddings = []
    chunk_payload = chunk_words(payload, 400)
    for i, chunk in enumerate(chunk_payload):
        #print("Chunk ",i)
        #print("Content ",chunk)
        embeddings_chunk = get_embedding(chunk, runtime_client,embed_llm_id)
        embeddings.append(embeddings_chunk)
    return embeddings

In [19]:
from sklearn.preprocessing import normalize
import numpy as np 

def parse_response(query_response, runtime_client,embed_llm_id):
    """Parse response and return the embedding."""
    embeddings = np.array(query_endpoint(query_response, runtime_client, embed_llm_id))
    #avg_embeddings = np.mean(embeddings, axis=0)
    # try max pooling of embedding vector
    avg_embeddings = np.max(embeddings, axis=0)

    avg_embeddings = avg_embeddings.reshape(1, -1)
    # normalization before inner product
    avg_embeddings = normalize(avg_embeddings, axis=1, norm='l2')
    return np.squeeze(avg_embeddings)

In [20]:
embedding = parse_response(payload, runtime_client, embed_llm_id)

In [21]:
print(embedding)
print(abs(np.linalg.norm(embedding) - 1) < 1e-5)

[ 0.06288187 -0.0086542  -0.02921784 ...  0.0128622   0.05716533
 -0.00952756]
True


#### Now we will pass all content for embedding

In [22]:
# Define a name for the output file
pdf_json_file_name = './data/content.json'

with open(pdf_json_file_name, 'r') as f:
    data = json.load(f)

In [24]:
# Loop through pages
for page_index, page in enumerate(data):
    if page_index % 100 == 0:
        print(f'Processing page {page_index}')
    data[page_index]["embedding"] = parse_response(data[page_index]["content"], runtime_client, embed_llm_id).tolist()

Processing page 0
Processing page 100
Processing page 200
Processing page 300


In [25]:
output_file_name = './data/embedding.json'
with open(output_file_name, 'w') as f:
    # Use json.dump to write pdfText to the file
    json.dump(data, f, indent=4)